In [1]:
import pandas as pd
import numpy as np

In [2]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

## Word Embeddings Tutorial

https://www.tensorflow.org/text/guide/word_embeddings

In [3]:
# download imdb dataset
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84140032/84125825 [==============================] - 46s 1us/step


['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [5]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [6]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [8]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print(f'label:{label_batch[i].numpy()}, review: {text_batch.numpy()[i]}\n')

label:0, review: b"Wow. Some movies just leave me speechless. This was undeniably one of those movies. When I left the theatre, not a single word came to my mouth. All I had was an incredible urge to slam my head against the theatre wall to help me forget about the last hour and a half. Unfortunately, it didn't work. Honestly, this movie has nothing to recommend. The humor was at the first grade level, at best, the acting was overly silly, and the plot was astronomically far-fetched. I hearby pledge never to see an other movie starring Chris Kattan or any other cast-member of SNL."

label:1, review: b'If any show in the last ten years deserves a 10, it is this rare gem. It allows us to escape back to a time when things were simpler and more fun. Filled with heart and laughs, this show keeps you laughing through the three decades of difference. The furniture was ugly, the clothes were colorful, and the even the drugs were tolerable. The hair was feathered, the music was accompanied by r

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [11]:
# result will have one additional dimension (embedding dimension)
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.02979015, -0.02164745, -0.02146624,  0.03158761, -0.00946365],
       [-0.02562147,  0.00032326, -0.04410921,  0.00671845,  0.02911177],
       [-0.01744045, -0.00072584,  0.03364461,  0.04117045,  0.00599056]],
      dtype=float32)

In [14]:
inpt = tf.constant([[0, 1, 2], [3, 4, 5]])
print(f'input shape: {inpt.shape}')
result = embedding_layer(inpt)
print(f'output shape: {result.shape}')

input shape: (2, 3)
output shape: (2, 3, 5)


In [15]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation), '')

# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [16]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [17]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 14s 687ms/step - loss: 0.6922 - accuracy: 0.5028 - val_loss: 0.6906 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 3s 148ms/step - loss: 0.6882 - accuracy: 0.5028 - val_loss: 0.6855 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 3s 141ms/step - loss: 0.6813 - accuracy: 0.5028 - val_loss: 0.6771 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 3s 146ms/step - loss: 0.6701 - accuracy: 0.5028 - val_loss: 0.6635 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 3s 134ms/step - loss: 0.6530 - accuracy: 0.5028 - val_loss: 0.6453 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 3s 144ms/step - loss: 0.6307 - accuracy: 0.5033 - val_loss: 0.6222 - val_accuracy: 0.4948
Epoch 7/15
20/20 [==============================] - 3s 129ms/step - loss: 0.6033 - accuracy: 0.5516 - val_loss: 0.5956 - val_accuracy: 0.5778
Epoch

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [21]:
%load_ext tensorboard
%tensorboard --logdir logs

In [22]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [23]:
# write the weights to disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue  # skip 0, it's padding.
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

## Basic Text Classification Tutorial

In [24]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing